<a href="https://colab.research.google.com/github/meituan/YOLOv6/blob/main/turtorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This is the official YOLOv6 notebook by MeiTuan, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/meituan/YOLOv6. Thank you!

# Introduction

YOLOv6 is a single-stage object detection framework dedicated to industrial applications, with hardware-friendly efficient design and high performance.

YOLOv6 is composed of the following methods:

Hardware-friendly Design for Backbone and Neck
Efficient Decoupled Head with SIoU Loss

# Setup
Clone repo and install dependencies.

In [ ]:
!git clone https://github.com/meituan/YOLOv6.git
%cd YOLOv6
%pip install -r requirements.txt

Cloning into 'YOLOv6'...
remote: Enumerating objects: 1807, done.
remote: Counting objects: 100% (821/821), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1807 (delta 630), reused 697 (delta 592), pack-reused 986
Receiving objects: 100% (1807/1807), 16.60 MiB | 5.25 MiB/s, done.
Resolving deltas: 100% (994/994), done.
/content/YOLOv6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 21.0 MB/s 
     |████████████████████████████████| 2.0 MB 43.5 MB/s 
ERROR: Could not find a version that satisfies the requirement pytorch_quantization>=2.1.1 (from versions: 0.0.1.dev4, 0.0.1.dev5)
ERROR: No matching distribution found for pytorch_quantization>=2.1.1


# Inference
First, download a pretrained model from the YOLOv6 [release](https://github.com/meituan/YOLOv6/releases).



In [1]:
# Download a pretrained model
import torch
torch.hub.download_url_to_file('https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6s.pt', 'yolov6s.pt')

100%|██████████| 38.9M/38.9M [00:02<00:00, 16.0MB/s]


Second, run inference with `tools/infer.py`, and saving results to `runs/inference`. Example inference sources are:

```shell
python tools/infer.py --weights yolov6s.pt --source img.jpg / imgdir
                                yolov6n.pt
```

In [4]:
!python tools/infer.py --weights yolov6s.pt --source data/images/image1.jpg
# show image
import cv2
img = cv2.imread('runs/inference/exp/image1.jpg')
#cv2_imshow(img)

Namespace(agnostic_nms=False, classes=None, conf_thres=0.4, device='0', half=False, hide_conf=False, hide_labels=False, img_size=[640, 640], iou_thres=0.45, max_det=1000, name='exp', not_save_img=False, project='runs/inference', save_dir=None, save_txt=False, source='data/images/image1.jpg', view_img=False, webcam=False, webcam_addr='0', weights='yolov6s.pt', yaml='data/coco.yaml')
Save directory already existed
checkpoint yolov6s.pt not exist, try to downloaded it from github.
downloading url is: https://github.com/meituan/YOLOv6/releases/download/0.4.0/yolov6s.pt, pealse make sure the version of the downloading model is correspoing to the code version!
checkpoint yolov6s.pt downloaded and saved
Loading checkpoint from yolov6s.pt

Fusing model...
Switch model to deploy modality.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.57it/s]
Results saved to runs/inference/exp


### You can also use torch.hub style to load the pretrained model or custom model to inference.

In [6]:
import torch 

model_local = torch.hub.load('.', 'yolov6n', source='local')  

Loading checkpoint from /DATA2/dse313/group14/YOLOv6/weights/yolov6n.pt

Fusing model...
/DATA2/dse313/group14/virtual_env/lib/python3.8/site-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  if param.grad is not None:


In [7]:
img_path = 'data/images/image1.jpg'

In [8]:
prediction = model_local.predict(img_path)
#prediction = model_custom.predict(img_path)
display(prediction)

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [9]:
model_local.show_predict(img_path)

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

# Validate
Validate a model's accuracy on [COCO](https://cocodataset.org/#home) val or test-dev datasets. Models are downloaded automatically from the [latest YOLOv6 release](https://github.com/meituan/YOLOv6/releases). 

## COCO val
Download COCO val 2017 dataset (1GB - 5000 images), and test model accuracy.

In [10]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

100%|██████████| 780M/780M [01:05<00:00, 12.5MB/s] 


In [11]:
# Run yolov6x on coco val
!python tools/eval.py --weights yolov6s.pt --data data/coco.yaml --img 640

Namespace(batch_size=32, conf_thres=0.03, config_file='', data='data/coco.yaml', device='0', do_coco_metric=True, do_pr_metric=False, eval_config_file='./configs/experiment/eval_640_repro.py', half=False, height=None, img_size=640, infer_on_rect=True, iou_thres=0.65, name='exp', plot_confusion_matrix=False, plot_curve=True, reproduce_640_eval=False, save_dir='runs/val/', shrink_size=0, specific_shape=False, task='val', verbose=False, weights='yolov6s.pt', width=None)
Loading checkpoint from yolov6s.pt

Fusing model...
Switch model to deploy modality.
Model Summary: Params: 18.54M, Gflops: 45.28
img record infomation path is:../coco/images/.val2017_cache.json
Val: Checking formats of images with 8 process(es): 
0 image(s) corrupted: 100%|██████████████| 5000/5000 [00:00<00:00, 20772.11it/s]
Val: Checking formats of labels with 8 process(es): 
4952 label(s) found, 48 label(s) missing, 0 label(s) empty, 0 invalid label file
Val: Final numbers of valid images: 5000/ labels: 5000. 
1.8s for

# Train coco data
conf: select config file to specify network/optimizer/hyperparameters

data: prepare [COCO](http://cocodataset.org)  dataset, [YOLO format coco labes](https://github.com/meituan/YOLOv6/releases/download/0.1.0/coco2017labels.zip) and specify dataset paths in data.yaml

make sure your dataset structure as fellows:
```shell
├── coco
│   ├── annotations
│   │   ├── instances_train2017.json
│   │   └── instances_val2017.json
│   ├── images
│   │   ├── train2017
│   │   └── val2017
│   ├── labels
│   │   ├── train2017
│   │   ├── val2017
│   ├── LICENSE
│   ├── README.txt
```

## COCO datasets

In [ ]:
# Download coco datasets and need about 30mins.
%cd ..
%cd coco/images
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip
!unzip train2017.zip && rm train2017.zip
!unzip val2017.zip && rm val2017.zip
!unzip test2017.zip && rm test2017.zip

In [ ]:
# Before running, you need to make sure you're in the YOLOv6 root directory.
%cd ../../YOLOv6
# Train YOLOv6s on COCO for 30 epochs
!python tools/train.py --img 640 --batch 32 --epochs 30 --conf configs/yolov6s.py --data data/coco.yaml

## COCO128 datasets
You need create a new file `coco128.yaml` under the folder `./data`.The details are as follows:

```
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../coco128  # dataset root dir
train: images/train2017  # train images (relative to 'path') 128 images
val: images/train2017  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 80  # number of classes
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']  # class names
```

In [ ]:
coco128 = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../coco128  # dataset root dir
train: ../coco128/images/train2017  # train images 128 images
val: ../coco128/images/train2017  # val images 128 images
test:  # test images (optional)

# Classes
nc: 80  # number of classes
names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']  # class names
"""

with open('data/coco128.yaml', 'w') as f:
  f.write(coco128)

In [ ]:
# Download coco128 datasets
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco128.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

# torch.hub.download_url_to_file('https://drive.google.com/file/d/1HICm-rrsdp89GNpFbzcwksHRtDx10McK/view?usp=sharing', 'tmp.zip')
# !unzip -q tmp.zip -d ../ && rm tmp.zip

  0%|          | 0.00/6.66M [00:00<?, ?B/s]

In [ ]:
# Train YOLOv6s on COCO128 for 100 epochs
!python tools/train.py --img 640 --batch 32 --epochs 100 --data data/coco128.yaml

Using 1 GPU for training... 
training args are: Namespace(batch_size=32, check_images=False, check_labels=False, conf_file='./configs/yolov6s.py', data_path='data/coco128.yaml', device='0', dist_url='env://', epochs=100, eval_final_only=False, eval_interval=20, gpu_count=0, heavy_eval_range=50, img_size=640, local_rank=-1, name='exp', output_dir='./runs/train', rank=-1, resume=False, save_dir='runs/train/exp', workers=8, world_size=1)

Train: Checking formats of images with 2 process(es): 

  0% 0/128 [00:00<?, ?it/s]
0 image(s) corrupted: 100% 128/128 [00:00<00:00, 3223.54it/s]
Train: Checking formats of labels with 2 process(es): 
128 label(s) found, 0 label(s) missing, 2 label(s) empty, 0 invalid label files: 100% 128/128 [00:00<00:00, 3653.13it/s]
Train: Final numbers of valid images: 128/ labels: 128. 
0.2s for dataset initialization.
Convert to COCO format
100% 128/128 [00:00<00:00, 32588.98it/s]
Convert to COCO format finished. Results saved in ../coco128/annotations/instances_t

In [ ]:
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

# Train Custom Data
This guidence explains how to train your own custom data with YOLOv6 (take fine-tuning YOLOv6-s model for example).

## Prepare your own dataset

**Step 1** Prepare your own dataset with images. For labeling images, you can use tools like [Labelme](https://github.com/wkentaro/labelme).

**Step 2** Generate label files in YOLO format.

One image corresponds to one label file, and the label format example is presented as below.

```json
# class_id center_x center_y bbox_width bbox_height
0 0.300926 0.617063 0.601852 0.765873
1 0.575 0.319531 0.4 0.551562
```

- Each row represents one object.
- Class id starts from `0`.
- Boundingbox coordinates must be in normalized `xywh` format (from 0 - 1). If your boxes are in pixels, divide `center_x` and `bbox_width` by image width, and `center_y` and `bbox_height` by image height.

**Step 3** Organize directories.

Organize your directory of custom dataset as follows:

```shell
custom_dataset
├── images
│   ├── train
│   │   ├── train0.jpg
│   │   └── train1.jpg
│   ├── val
│   │   ├── val0.jpg
│   │   └── val1.jpg
│   └── test
│       ├── test0.jpg
│       └── test1.jpg
└── labels
    ├── train
    │   ├── train0.txt
    │   └── train1.txt
    ├── val
    │   ├── val0.txt
    │   └── val1.txt
    └── test
        ├── test0.txt
        └── test1.txt
```

**Step 4** Create `dataset.yaml` in `$YOLOv6_DIR/data`.

```yaml
# Please insure that your custom_dataset are put in same parent dir with YOLOv6_DIR
train: ../custom_dataset/images/train # train images
val: ../custom_dataset/images/val # val images
test: ../custom_dataset/images/test # test images (optional)

# whether it is coco dataset, only coco dataset should be set to True.
is_coco: False

# Classes
nc: 20  # number of classes
names: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
        'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']  # class names
```

## Create a config file


We use a config file to specify the network structure and training setting, including  optimizer and data augmentation hyperparameters.

If you create a new config file, please put it under the configs directory.
Or just use the provided config file in `$YOLOV6_HOME/configs/*_finetune.py`.

```python
## YOLOv6s Model config file
model = dict(
    type='YOLOv6s',
    pretrained='./weights/yolov6s.pt', # download pretrain model from YOLOv6 github if use pretrained model
    depth_multiple = 0.33,
    width_multiple = 0.50,
    ...
)
solver=dict(
    optim='SGD',
    lr_scheduler='Cosine',
    ...
)

data_aug = dict(
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    ...
)
```



## Train

In [32]:
import torch.cuda

# print(torch.cuda.device_count())
# print(torch.cuda.current_device())
!python tools/train.py --batch 256 --conf configs/yolov6s_finetune.py --data data/dataset.yaml

-----------------------------------------------------
2
2
Using 1 GPU for training... 
training args are: Namespace(batch_size=256, bs_per_gpu=32, cache_ram=False, calib=False, check_images=False, check_labels=False, conf_file='configs/yolov6s_finetune.py', data_path='data/dataset.yaml', device='2', dist_url='env://', distill=False, distill_feat=False, epochs=400, eval_final_only=False, eval_interval=20, fuse_ab=False, gpu_count=3, heavy_eval_range=50, height=None, img_size=640, local_rank=-1, name='exp', output_dir='./runs/train', quant=False, rank=-1, rect=False, resume=False, save_ckpt_on_last_n_epoch=-1, save_dir='runs/train/exp20', specific_shape=False, stop_aug_last_n_epoch=15, teacher_model_path=None, temperature=20, width=None, workers=8, world_size=1, write_trainbatch_tb=False)

Loading state_dict from weights/yolov6s.pt for fine-tuning...
Model: Model(
  (backbone): EfficientRep(
    (stem): RepVGGBlock(
      (nonlinearity): ReLU(inplace=True)
      (se): Identity()
      (r

# Test Speed

In [ ]:
!python tools/eval.py --data data/coco128.yaml --batch 32 --weights yolov6s.pt --task speed

Namespace(batch_size=32, conf_thres=0.001, data='data/coco128.yaml', device='0', half=False, img_size=640, iou_thres=0.65, name='exp', save_dir='runs/val/', task='speed', weights='yolov6s.pt')
Loading checkpoint from yolov6s.pt

Fusing model...
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Switch model to deploy modality.
Model Summary: Params: 17.22M, Gflops: 44.19
Speed: Checking formats of labels with 2 process(es): 
128 label(s) found, 0 label(s) missing, 2 label(s) empty, 0 invalid label files: 100% 128/128 [00:00<00:00, 2462.39it/s]
Speed: Final numbers of valid images: 128/ labels: 128. 
0.2s for dataset initialization.
Inferencing model in val datasets.: 100% 4/4 [00:01<00:00,  2.24it/s]

Evaluating speed.
Average pr